# content
- [web scraping](#web_scraping)
- [data cleaning](#data_cleaning)
- [data analysis](#data_analysis)
- [conclusions](#conclusions)

In [ ]:
# importing libraries
from bs4 import BeautifulSoup
import requests
import datetime
import csv
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math
import re as re
import os
import psycopg2 as ps
import sqlite3

## project scenario
- we are a startup starting a business in the kitchen appliances industry, we need to know **our main competitors** , **the most succeful products in each category** , and the **characteristics and pricing of the most succefull products.**
- the company is willing to produce products in 7 main categories: air fryers, food processors, espresso machines, blenders, mixers, toasters, and electric kettles.
- in order to answer the above questions we will scrape amazon data about the products with +4 stars rating in each of these catogeries and analyze their data to give a reliable answer to each one of them.

# web_scraping

In this part I create the main file which we will use in the data cleaning and analysis, we scrape each category by providing its link and scrape its products' details using the append_product function. <br>
I commented the lines that will create a new file just for myself so that the file doesn't get deleted.

In [ ]:
'''
main_page='https://www.amazon.eg/s?i=home&bbn=21863949031&rh=n%3A18021933031%2Cn%3A21863792031%2Cn%3A21863827031%2Cn%3A21863949031%2Cn%3A26957429031%2Cp_72%3A21909187031&dc&fs=true&language=en&ds=v1%3AAU%2FON3FdtLe4umfE6xf%2FVGlQf3BC%2B4wHfqH7eQw3qI0&qid=1673216837&rnid=21863949031&ref=sr_nr_n_1'
mains=[main_page]
filename='amazonKitchenFullData.csv'
'''

In [ ]:
'''
# creating the file and its headers
header=['title','brand','category','price','date','rating', 'num_of_ratings','link']

with open(filename,'w',newline='',encoding='UTF8') as f:
    writer=csv.writer(f)
    writer.writerow(header)
'''

In [ ]:
links=[]
titles=[]
def append_links(url):
    '''
    this function takes the page link as argument and returns all the products links in that page. 
    '''
    headers= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36"}
    page=requests.get(url,headers=headers,cookies={'lc-acbeg':'en_AE'},timeout=30,verify=False)
    soup1= BeautifulSoup(page.content,'html.parser')
    soup2=BeautifulSoup(soup1.prettify(),'html.parser')
    for link in soup2.find_all('a',attrs={"class": "a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"}):
        links.append('https://www.amazon.eg'+link.get('href'))
    for title in soup2.find_all('span',attrs={"class": "a-size-medium a-color-base a-text-normal"}):
        titles.append(title.get_text().strip())

In [ ]:
for x in range(0,4): # the range is the number of pages
    headers= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36"}
    page=requests.get(main_page,headers=headers,cookies={'lc-acbeg':'en_AE'},timeout=30,verify=False)
    soup1= BeautifulSoup(page.content,'html.parser')
    soup2=BeautifulSoup(soup1.prettify(),'html.parser')
    try:
        main2=soup2.find('a',attrs={"class": "s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"}).get('href')
    except:
        main2=main_page
    if main2 in mains:
        append_links(main_page)
        break
    else:
        mains.append('https://www.amazon.eg'+main2)
    append_links(main_page)
    main_page='https://www.amazon.eg'+main2

In [ ]:
# checking the length and the links
print(len(mains))
print(mains[-5:])

In [ ]:
print(titles[-5:])
print(len(titles))
print(links[-5:])
print(len(links))

In [ ]:
missingLinks=[]
def appendProduct(link):
    '''
    this function takes the product link as an argument and returns the title, the price, category, rating, number of ratings, date and the link of the product.
    '''
    headers= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36"}
    page=requests.get(link,headers=headers,cookies={'lc-acbeg':'en_AE'},timeout=30,verify=False)
    soup1= BeautifulSoup(page.content,'html.parser')
    soup2=BeautifulSoup(soup1.prettify(),'html.parser')
    title=titles[i]
    try:
        brand=soup2.find("a", {"id": "bylineInfo"}).get_text().strip()
    except AttributeError:
        try:
            link=links[i]
            page = requests.get(link,headers=headers,verify=False, timeout=30)
            soup1 = BeautifulSoup(page.content,'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
            brand=soup2.find("a", {"id": "bylineInfo"}).get_text().strip()
        except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                brand=soup2.find("a", {"id": "bylineInfo"}).get_text().strip()
            except AttributeError:
                brand='N/A'
    try:
        price=soup2.find("span", {"class": "a-price-whole"}).get_text().strip()
    except AttributeError:
        try:
            link=links[i]
            page = requests.get(link,headers=headers,verify=False, timeout=30)
            soup1 = BeautifulSoup(page.content,'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
            price=soup2.find("span", {"class": "a-price-whole"}).get_text().strip()
        except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                price=soup2.find("span", {"class": "a-price-whole"}).get_text().strip()
            except AttributeError:
                price='N/A'
    try:
        num_of_ratings=soup2.find("span", {"id": "acrCustomerReviewText"}).get_text().strip()
    except AttributeError:
        try:
            link=links[i]
            page = requests.get(link,headers=headers,verify=False, timeout=30)
            soup1 = BeautifulSoup(page.content,'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
            num_of_ratings=soup2.find("span", {"id": "acrCustomerReviewText"}).get_text().strip()
        except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                num_of_ratings=soup2.find("span", {"id": "acrCustomerReviewText"}).get_text().strip()
            except AttributeError:
                num_of_ratings='N/A'

    today=datetime.date.today()
    try:
        rating=soup2.find("span", {"class": "a-icon-alt"}).get_text().strip()
        if rating=='Previous page':
            rating='N/A'
        else:
            rating=rating
    except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                rating=soup2.find("span", {"class": "a-icon-alt"}).get_text().strip()
                if rating=='Previous page':
                    rating='N/A'
                else:
                    rating=rating
            except AttributeError:
                rating='N/A'
    if rating == 'N/A' and price == 'N/A':
        missingLinks.append(links[i])
    link=links[i]
    category='air fryers'
    data=[title,brand,category,price,today,rating, num_of_ratings,link]
    with open(filename,'a+',newline='',encoding='UTF8') as f:
        writer=csv.writer(f)
        writer.writerow(data)


In [ ]:
# loop over products
i=0
for link in links:
    appendProduct(links[i])
    i+=1

In [ ]:
print(i)

we start repeating the process for each category until we're done

In [ ]:
main_page='https://www.amazon.eg/s?i=home&bbn=21863955031&rh=n%3A18021933031%2Cn%3A21863792031%2Cn%3A21863827031%2Cn%3A21863955031%2Cn%3A21864402031%2Cp_72%3A21909187031&dc&fs=true&language=en&ds=v1%3AaEqB9e39uL4q4gv8EZmR7Pzm9d8qJI8AIhkoKcbetaM&qid=1673219711&rnid=21863955031&ref=sr_nr_n_2'
mains=[main_page]

In [ ]:
links=[]
titles=[]
for x in range(0,5): # the range is the number of pages
    headers= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36"}
    page=requests.get(main_page,headers=headers,cookies={'lc-acbeg':'en_AE'},timeout=30,verify=False)
    soup1= BeautifulSoup(page.content,'html.parser')
    soup2=BeautifulSoup(soup1.prettify(),'html.parser')
    try:
        main2=soup2.find('a',attrs={"class": "s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"}).get('href')
    except:
        main2=main_page
    if main2 in mains:
        append_links(main_page)
        break
    else:
        mains.append('https://www.amazon.eg'+main2)
    append_links(main_page)
    main_page='https://www.amazon.eg'+main2

In [ ]:
print(len(mains))
print(mains[-5:])
print(titles[-5:])
print(len(titles))
print(links[-5:])
print(len(links))

In [ ]:
missingLinks=[]
def appendProduct(link):
    '''
    this function takes the product link as an argument and returns the title, the price, category, rating, number of ratings, date and the link of the product.
    '''
    headers= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36"}
    page=requests.get(link,headers=headers,cookies={'lc-acbeg':'en_AE'},timeout=30,verify=False)
    soup1= BeautifulSoup(page.content,'html.parser')
    soup2=BeautifulSoup(soup1.prettify(),'html.parser')
    title=titles[i]
    try:
        brand=soup2.find("a", {"id": "bylineInfo"}).get_text().strip()
    except AttributeError:
        try:
            link=links[i]
            page = requests.get(link,headers=headers,verify=False, timeout=30)
            soup1 = BeautifulSoup(page.content,'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
            brand=soup2.find("a", {"id": "bylineInfo"}).get_text().strip()
        except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                brand=soup2.find("a", {"id": "bylineInfo"}).get_text().strip()
            except AttributeError:
                brand='N/A'
    try:
        price=soup2.find("span", {"class": "a-price-whole"}).get_text().strip()
    except AttributeError:
        try:
            link=links[i]
            page = requests.get(link,headers=headers,verify=False, timeout=30)
            soup1 = BeautifulSoup(page.content,'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
            price=soup2.find("span", {"class": "a-price-whole"}).get_text().strip()
        except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                price=soup2.find("span", {"class": "a-price-whole"}).get_text().strip()
            except AttributeError:
                price='N/A'
    try:
        num_of_ratings=soup2.find("span", {"id": "acrCustomerReviewText"}).get_text().strip()
    except AttributeError:
        try:
            link=links[i]
            page = requests.get(link,headers=headers,verify=False, timeout=30)
            soup1 = BeautifulSoup(page.content,'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
            num_of_ratings=soup2.find("span", {"id": "acrCustomerReviewText"}).get_text().strip()
        except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                num_of_ratings=soup2.find("span", {"id": "acrCustomerReviewText"}).get_text().strip()
            except AttributeError:
                num_of_ratings='N/A'

    today=datetime.date.today()
    try:
        rating=soup2.find("span", {"class": "a-icon-alt"}).get_text().strip()
        if rating=='Previous page':
            rating='N/A'
        else:
            rating=rating
    except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                rating=soup2.find("span", {"class": "a-icon-alt"}).get_text().strip()
                if rating=='Previous page':
                    rating='N/A'
                else:
                    rating=rating
            except AttributeError:
                rating='N/A'
    if rating == 'N/A' and price == 'N/A':
        missingLinks.append(links[i])
    link=links[i]
    category='food processors'
    data=[title,brand,category,price,today,rating, num_of_ratings,link]
    with open(filename,'a+',newline='',encoding='UTF8') as f:
        writer=csv.writer(f)
        writer.writerow(data)


In [ ]:
# loop over products
i=0
for link in links:
    appendProduct(links[i])
    i+=1

In [ ]:
print(i)

In [ ]:
main_page='https://www.amazon.eg/s?i=home&bbn=21864088031&rh=n%3A18021933031%2Cn%3A21863792031%2Cn%3A21863827031%2Cn%3A21864088031%2Cn%3A21864618031%2Cp_72%3A21909187031&dc&fs=true&language=en&ds=v1%3ATjaYlZt6%2BQiKZCN9dV8Uiwl84heZskiWDf54xFFmw6E&qid=1673220410&rnid=21864088031&ref=sr_nr_n_2'
mains=[main_page]

In [ ]:

links=[]
titles=[]
for x in range(0,4): # the range is the number of pages
    headers= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36"}
    page=requests.get(main_page,headers=headers,cookies={'lc-acbeg':'en_AE'},timeout=30,verify=False)
    soup1= BeautifulSoup(page.content,'html.parser')
    soup2=BeautifulSoup(soup1.prettify(),'html.parser')
    try:
        main2=soup2.find('a',attrs={"class": "s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"}).get('href')
    except:
        main2=main_page
    if main2 in mains:
        append_links(main_page)
        break
    else:
        mains.append('https://www.amazon.eg'+main2)
    append_links(main_page)
    main_page='https://www.amazon.eg'+main2


In [ ]:
print(len(mains))
print(mains[-5:])
print(titles[-5:])
print(len(titles))
print(links[-5:])
print(len(links))

In [ ]:
missingLinks=[]
def appendProduct(link):
    '''
    this function takes the product link as an argument and returns the title, the price, category, rating, number of ratings, date and the link of the product.
    '''
    headers= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36"}
    page=requests.get(link,headers=headers,cookies={'lc-acbeg':'en_AE'},timeout=30,verify=False)
    soup1= BeautifulSoup(page.content,'html.parser')
    soup2=BeautifulSoup(soup1.prettify(),'html.parser')
    title=titles[i]
    try:
        brand=soup2.find("a", {"id": "bylineInfo"}).get_text().strip()
    except AttributeError:
        try:
            link=links[i]
            page = requests.get(link,headers=headers,verify=False, timeout=30)
            soup1 = BeautifulSoup(page.content,'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
            brand=soup2.find("a", {"id": "bylineInfo"}).get_text().strip()
        except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                brand=soup2.find("a", {"id": "bylineInfo"}).get_text().strip()
            except AttributeError:
                brand='N/A'
    try:
        price=soup2.find("span", {"class": "a-price-whole"}).get_text().strip()
    except AttributeError:
        try:
            link=links[i]
            page = requests.get(link,headers=headers,verify=False, timeout=30)
            soup1 = BeautifulSoup(page.content,'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
            price=soup2.find("span", {"class": "a-price-whole"}).get_text().strip()
        except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                price=soup2.find("span", {"class": "a-price-whole"}).get_text().strip()
            except AttributeError:
                price='N/A'
    try:
        num_of_ratings=soup2.find("span", {"id": "acrCustomerReviewText"}).get_text().strip()
    except AttributeError:
        try:
            link=links[i]
            page = requests.get(link,headers=headers,verify=False, timeout=30)
            soup1 = BeautifulSoup(page.content,'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
            num_of_ratings=soup2.find("span", {"id": "acrCustomerReviewText"}).get_text().strip()
        except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                num_of_ratings=soup2.find("span", {"id": "acrCustomerReviewText"}).get_text().strip()
            except AttributeError:
                num_of_ratings='N/A'

    today=datetime.date.today()
    try:
        rating=soup2.find("span", {"class": "a-icon-alt"}).get_text().strip()
        if rating=='Previous page':
            rating='N/A'
        else:
            rating=rating
    except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                rating=soup2.find("span", {"class": "a-icon-alt"}).get_text().strip()
                if rating=='Previous page':
                    rating='N/A'
                else:
                    rating=rating
            except AttributeError:
                rating='N/A'
    if rating == 'N/A' and price == 'N/A':
        missingLinks.append(links[i])
    link=links[i]
    category='espresso machines'
    data=[title,brand,category,price,today,rating, num_of_ratings,link]
    with open(filename,'a+',newline='',encoding='UTF8') as f:
        writer=csv.writer(f)
        writer.writerow(data)


In [ ]:
# loop over products
i=0
for link in links:
    appendProduct(links[i])
    i+=1

In [ ]:
print(i)

In [ ]:
main_page='https://www.amazon.eg/s?i=home&bbn=21864401031&rh=n%3A18021933031%2Cn%3A21863792031%2Cn%3A21863827031%2Cn%3A21863955031%2Cn%3A21864401031%2Cp_72%3A21909187031&dc&fs=true&language=en&ds=v1%3AJY%2BuJtvPkJ4eeU3gvw9f3%2B9urALxpLDUzdaJI9hn0nc&qid=1673220767&ref=sr_ex_n_1'
mains=[main_page]

In [ ]:
links=[]
titles=[]
for x in range(0,13): # the range is the number of pages
    headers= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36"}
    page=requests.get(main_page,headers=headers,cookies={'lc-acbeg':'en_AE'},timeout=30,verify=False)
    soup1= BeautifulSoup(page.content,'html.parser')
    soup2=BeautifulSoup(soup1.prettify(),'html.parser')
    try:
        main2=soup2.find('a',attrs={"class": "s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"}).get('href')
    except:
        main2=main_page
    if main2 in mains:
        append_links(main_page)
        break
    else:
        mains.append('https://www.amazon.eg'+main2)
    append_links(main_page)
    main_page='https://www.amazon.eg'+main2


In [ ]:
print(len(mains))
print(mains[-5:])
print(titles[-5:])
print(len(titles))
print(links[-5:])
print(len(links))

In [ ]:
missingLinks=[]
def appendProduct(link):
    '''
    this function takes the product link as an argument and returns the title, the price, category, rating, number of ratings, date and the link of the product.
    '''
    headers= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36"}
    page=requests.get(link,headers=headers,cookies={'lc-acbeg':'en_AE'},timeout=30,verify=False)
    soup1= BeautifulSoup(page.content,'html.parser')
    soup2=BeautifulSoup(soup1.prettify(),'html.parser')
    title=titles[i]
    try:
        brand=soup2.find("a", {"id": "bylineInfo"}).get_text().strip()
    except AttributeError:
        try:
            link=links[i]
            page = requests.get(link,headers=headers,verify=False, timeout=30)
            soup1 = BeautifulSoup(page.content,'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
            brand=soup2.find("a", {"id": "bylineInfo"}).get_text().strip()
        except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                brand=soup2.find("a", {"id": "bylineInfo"}).get_text().strip()
            except AttributeError:
                brand='N/A'
    try:
        price=soup2.find("span", {"class": "a-price-whole"}).get_text().strip()
    except AttributeError:
        try:
            link=links[i]
            page = requests.get(link,headers=headers,verify=False, timeout=30)
            soup1 = BeautifulSoup(page.content,'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
            price=soup2.find("span", {"class": "a-price-whole"}).get_text().strip()
        except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                price=soup2.find("span", {"class": "a-price-whole"}).get_text().strip()
            except AttributeError:
                price='N/A'
    try:
        num_of_ratings=soup2.find("span", {"id": "acrCustomerReviewText"}).get_text().strip()
    except AttributeError:
        try:
            link=links[i]
            page = requests.get(link,headers=headers,verify=False, timeout=30)
            soup1 = BeautifulSoup(page.content,'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
            num_of_ratings=soup2.find("span", {"id": "acrCustomerReviewText"}).get_text().strip()
        except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                num_of_ratings=soup2.find("span", {"id": "acrCustomerReviewText"}).get_text().strip()
            except AttributeError:
                num_of_ratings='N/A'

    today=datetime.date.today()
    try:
        rating=soup2.find("span", {"class": "a-icon-alt"}).get_text().strip()
        if rating=='Previous page':
            rating='N/A'
        else:
            rating=rating
    except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                rating=soup2.find("span", {"class": "a-icon-alt"}).get_text().strip()
                if rating=='Previous page':
                    rating='N/A'
                else:
                    rating=rating
            except AttributeError:
                rating='N/A'
    if rating == 'N/A' and price == 'N/A':
        missingLinks.append(links[i])
    link=links[i]
    category='blenders'
    data=[title,brand,category,price,today,rating, num_of_ratings,link]
    with open(filename,'a+',newline='',encoding='UTF8') as f:
        writer=csv.writer(f)
        writer.writerow(data)


In [ ]:
# loop over products
i=0
for link in links:
    appendProduct(links[i])
    i+=1

In [ ]:
print(i)

In [ ]:
main_page='https://www.amazon.eg/-/en/s?i=home&bbn=21863955031&rh=n%3A18021933031%2Cn%3A21863792031%2Cn%3A21863827031%2Cn%3A21863955031%2Cn%3A21864400031%2Cp_72%3A21909187031&dc&fs=true&language=en&qid=1673223137&rnid=21863955031&ref=sr_pg_1'
mains=[main_page]

In [ ]:

links=[]
titles=[]
for x in range(0,8): # the range is the number of pages
    headers= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36"}
    page=requests.get(main_page,headers=headers,cookies={'lc-acbeg':'en_AE'},timeout=30,verify=False)
    soup1= BeautifulSoup(page.content,'html.parser')
    soup2=BeautifulSoup(soup1.prettify(),'html.parser')
    try:
        main2=soup2.find('a',attrs={"class": "s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"}).get('href')
    except:
        main2=main_page
    if main2 in mains:
        append_links(main_page)
        break
    else:
        mains.append('https://www.amazon.eg'+main2)
    append_links(main_page)
    main_page='https://www.amazon.eg'+main2


In [ ]:
print(len(mains))
print(mains[-5:])
print(titles[-5:])
print(len(titles))
print(links[-5:])
print(len(links))

In [ ]:
missingLinks=[]
def appendProduct(link):
    '''
    this function takes the product link as an argument and returns the title, the price, category, rating, number of ratings, date and the link of the product.
    '''
    headers= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36"}
    page=requests.get(link,headers=headers,cookies={'lc-acbeg':'en_AE'},timeout=30,verify=False)
    soup1= BeautifulSoup(page.content,'html.parser')
    soup2=BeautifulSoup(soup1.prettify(),'html.parser')
    title=titles[i]
    try:
        brand=soup2.find("a", {"id": "bylineInfo"}).get_text().strip()
    except AttributeError:
        try:
            link=links[i]
            page = requests.get(link,headers=headers,verify=False, timeout=30)
            soup1 = BeautifulSoup(page.content,'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
            brand=soup2.find("a", {"id": "bylineInfo"}).get_text().strip()
        except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                brand=soup2.find("a", {"id": "bylineInfo"}).get_text().strip()
            except AttributeError:
                brand='N/A'
    try:
        price=soup2.find("span", {"class": "a-price-whole"}).get_text().strip()
    except AttributeError:
        try:
            link=links[i]
            page = requests.get(link,headers=headers,verify=False, timeout=30)
            soup1 = BeautifulSoup(page.content,'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
            price=soup2.find("span", {"class": "a-price-whole"}).get_text().strip()
        except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                price=soup2.find("span", {"class": "a-price-whole"}).get_text().strip()
            except AttributeError:
                price='N/A'
    try:
        num_of_ratings=soup2.find("span", {"id": "acrCustomerReviewText"}).get_text().strip()
    except AttributeError:
        try:
            link=links[i]
            page = requests.get(link,headers=headers,verify=False, timeout=30)
            soup1 = BeautifulSoup(page.content,'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
            num_of_ratings=soup2.find("span", {"id": "acrCustomerReviewText"}).get_text().strip()
        except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                num_of_ratings=soup2.find("span", {"id": "acrCustomerReviewText"}).get_text().strip()
            except AttributeError:
                num_of_ratings='N/A'

    today=datetime.date.today()
    try:
        rating=soup2.find("span", {"class": "a-icon-alt"}).get_text().strip()
        if rating=='Previous page':
            rating='N/A'
        else:
            rating=rating
    except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                rating=soup2.find("span", {"class": "a-icon-alt"}).get_text().strip()
                if rating=='Previous page':
                    rating='N/A'
                else:
                    rating=rating
            except AttributeError:
                rating='N/A'
    if rating == 'N/A' and price == 'N/A':
        missingLinks.append(links[i])
    link=links[i]
    category='mixers'
    data=[title,brand,category,price,today,rating, num_of_ratings,link]
    with open(filename,'a+',newline='',encoding='UTF8') as f:
        writer=csv.writer(f)
        writer.writerow(data)


In [ ]:
# loop over products
i=0
for link in links:
    appendProduct(links[i])
    i+=1

In [ ]:
print(i)

In [ ]:
main_page='https://www.amazon.eg/s?i=home&bbn=21863827031&rh=n%3A18021933031%2Cn%3A21863792031%2Cn%3A21863827031%2Cn%3A21863951031%2Cp_72%3A21909187031&dc&fs=true&language=en&ds=v1%3Amq%2FzHDwBf1%2Bh5tyMIW%2BCd8a%2BkQF5YZbexSfQJI6XQIY&qid=1673223458&rnid=21863827031&ref=sr_nr_n_14'
mains=[main_page]

In [ ]:

links=[]
titles=[]
for x in range(0,12): # the range is the number of pages
    headers= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36"}
    page=requests.get(main_page,headers=headers,cookies={'lc-acbeg':'en_AE'},timeout=30,verify=False)
    soup1= BeautifulSoup(page.content,'html.parser')
    soup2=BeautifulSoup(soup1.prettify(),'html.parser')
    try:
        main2=soup2.find('a',attrs={"class": "s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"}).get('href')
    except:
        main2=main_page
    if main2 in mains:
        append_links(main_page)
        break
    else:
        mains.append('https://www.amazon.eg'+main2)
    append_links(main_page)
    main_page='https://www.amazon.eg'+main2


In [ ]:
print(len(mains))
print(mains[-5:])
print(titles[-5:])
print(len(titles))
print(links[-5:])
print(len(links))

In [ ]:
missingLinks=[]
def appendProduct(link):
    '''
    this function takes the product link as an argument and returns the title, the price, category, rating, number of ratings, date and the link of the product.
    '''
    headers= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36"}
    page=requests.get(link,headers=headers,cookies={'lc-acbeg':'en_AE'},timeout=30,verify=False)
    soup1= BeautifulSoup(page.content,'html.parser')
    soup2=BeautifulSoup(soup1.prettify(),'html.parser')
    title=titles[i]
    try:
        brand=soup2.find("a", {"id": "bylineInfo"}).get_text().strip()
    except AttributeError:
        try:
            link=links[i]
            page = requests.get(link,headers=headers,verify=False, timeout=30)
            soup1 = BeautifulSoup(page.content,'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
            brand=soup2.find("a", {"id": "bylineInfo"}).get_text().strip()
        except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                brand=soup2.find("a", {"id": "bylineInfo"}).get_text().strip()
            except AttributeError:
                brand='N/A'
    try:
        price=soup2.find("span", {"class": "a-price-whole"}).get_text().strip()
    except AttributeError:
        try:
            link=links[i]
            page = requests.get(link,headers=headers,verify=False, timeout=30)
            soup1 = BeautifulSoup(page.content,'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
            price=soup2.find("span", {"class": "a-price-whole"}).get_text().strip()
        except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                price=soup2.find("span", {"class": "a-price-whole"}).get_text().strip()
            except AttributeError:
                price='N/A'
    try:
        num_of_ratings=soup2.find("span", {"id": "acrCustomerReviewText"}).get_text().strip()
    except AttributeError:
        try:
            link=links[i]
            page = requests.get(link,headers=headers,verify=False, timeout=30)
            soup1 = BeautifulSoup(page.content,'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
            num_of_ratings=soup2.find("span", {"id": "acrCustomerReviewText"}).get_text().strip()
        except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                num_of_ratings=soup2.find("span", {"id": "acrCustomerReviewText"}).get_text().strip()
            except AttributeError:
                num_of_ratings='N/A'

    today=datetime.date.today()
    try:
        rating=soup2.find("span", {"class": "a-icon-alt"}).get_text().strip()
        if rating=='Previous page':
            rating='N/A'
        else:
            rating=rating
    except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                rating=soup2.find("span", {"class": "a-icon-alt"}).get_text().strip()
                if rating=='Previous page':
                    rating='N/A'
                else:
                    rating=rating
            except AttributeError:
                rating='N/A'
    if rating == 'N/A' and price == 'N/A':
        missingLinks.append(links[i])
    link=links[i]
    category='toasters'
    data=[title,brand,category,price,today,rating, num_of_ratings,link]
    with open(filename,'a+',newline='',encoding='UTF8') as f:
        writer=csv.writer(f)
        writer.writerow(data)


In [ ]:
# loop over products
i=0
for link in links:
    appendProduct(links[i])
    i+=1

In [ ]:
print(i)

In [ ]:
main_page='https://www.amazon.eg/s?i=home&bbn=21863961031&rh=n%3A18021933031%2Cn%3A21863792031%2Cn%3A21863827031%2Cn%3A21863961031%2Cn%3A21864412031%2Cp_72%3A21909187031&dc&fs=true&language=en&ds=v1%3A9oO9DbQer7rzESCMmhf2GgkjaTRWsxAw%2B%2F1k4S7BqEk&qid=1673224930&rnid=21863961031&ref=sr_nr_n_1'
mains=[main_page]

In [ ]:

links=[]
titles=[]
for x in range(0,8): # the range is the number of pages
    headers= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36"}
    page=requests.get(main_page,headers=headers,cookies={'lc-acbeg':'en_AE'},timeout=30,verify=False)
    soup1= BeautifulSoup(page.content,'html.parser')
    soup2=BeautifulSoup(soup1.prettify(),'html.parser')
    try:
        main2=soup2.find('a',attrs={"class": "s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"}).get('href')
    except:
        main2=main_page
    if main2 in mains:
        append_links(main_page)
        break
    else:
        mains.append('https://www.amazon.eg'+main2)
    append_links(main_page)
    main_page='https://www.amazon.eg'+main2


In [ ]:
print(len(mains))
print(mains[-5:])
print(titles[-5:])
print(len(titles))
print(links[-5:])
print(len(links))

In [ ]:
missingLinks=[]
def appendProduct(link):
    '''
    this function takes the product link as an argument and returns the title, the price, category, rating, number of ratings, date and the link of the product.
    '''
    headers= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36"}
    page=requests.get(link,headers=headers,cookies={'lc-acbeg':'en_AE'},timeout=30,verify=False)
    soup1= BeautifulSoup(page.content,'html.parser')
    soup2=BeautifulSoup(soup1.prettify(),'html.parser')
    title=titles[i]
    try:
        brand=soup2.find("a", {"id": "bylineInfo"}).get_text().strip()
    except AttributeError:
        try:
            link=links[i]
            page = requests.get(link,headers=headers,verify=False, timeout=30)
            soup1 = BeautifulSoup(page.content,'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
            brand=soup2.find("a", {"id": "bylineInfo"}).get_text().strip()
        except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                brand=soup2.find("a", {"id": "bylineInfo"}).get_text().strip()
            except AttributeError:
                brand='N/A'
    try:
        price=soup2.find("span", {"class": "a-price-whole"}).get_text().strip()
    except AttributeError:
        try:
            link=links[i]
            page = requests.get(link,headers=headers,verify=False, timeout=30)
            soup1 = BeautifulSoup(page.content,'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
            price=soup2.find("span", {"class": "a-price-whole"}).get_text().strip()
        except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                price=soup2.find("span", {"class": "a-price-whole"}).get_text().strip()
            except AttributeError:
                price='N/A'
    try:
        num_of_ratings=soup2.find("span", {"id": "acrCustomerReviewText"}).get_text().strip()
    except AttributeError:
        try:
            link=links[i]
            page = requests.get(link,headers=headers,verify=False, timeout=30)
            soup1 = BeautifulSoup(page.content,'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
            num_of_ratings=soup2.find("span", {"id": "acrCustomerReviewText"}).get_text().strip()
        except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                num_of_ratings=soup2.find("span", {"id": "acrCustomerReviewText"}).get_text().strip()
            except AttributeError:
                num_of_ratings='N/A'

    today=datetime.date.today()
    try:
        rating=soup2.find("span", {"class": "a-icon-alt"}).get_text().strip()
        if rating=='Previous page':
            rating='N/A'
        else:
            rating=rating
    except AttributeError:
            try:
                link=links[i]
                page = requests.get(link,headers=headers,verify=False, timeout=30)
                soup1 = BeautifulSoup(page.content,'html.parser')
                soup2 = BeautifulSoup(soup1.prettify(),'html.parser')
                rating=soup2.find("span", {"class": "a-icon-alt"}).get_text().strip()
                if rating=='Previous page':
                    rating='N/A'
                else:
                    rating=rating
            except AttributeError:
                rating='N/A'
    if rating == 'N/A' and price == 'N/A':
        missingLinks.append(links[i])
    link=links[i]
    category='electric kettles'
    data=[title,brand,category,price,today,rating, num_of_ratings,link]
    with open(filename,'a+',newline='',encoding='UTF8') as f:
        writer=csv.writer(f)
        writer.writerow(data)


In [ ]:
# loop over products
i=0
for link in links:
    appendProduct(links[i])
    i+=1

# data_cleaning

in this part we clean the amazon data we scraped earlier and scrape the missing values if necessary to make the data ready for further analysis

In [ ]:
df=pd.read_csv('amazonKitchenFullData.csv')
df.sample(5)

In [ ]:
df.info()

there're about 20 missing values in the brand column, 26 in the price, 4 in rating and 5 in number of ratings, we need to scrape that data as it's almost about 10% of our data.

In [ ]:
df.link.duplicated().sum()

In [ ]:
df.nunique()

In [ ]:
df_missing_info=df.query('brand.isnull() | price.isnull() | rating.isnull() | num_of_ratings.isnull()')
df_missing_info.sample(5)

In [ ]:
'''
missing_links=list(df_missing_info.link)
len(missing_links)
'''

In [ ]:
'''
filename='amazonMissingItems.csv'
header=['title','brand','category','price','date','rating', 'num_of_ratings','link']

with open(filename,'w',newline='',encoding='UTF8') as f:
    writer=csv.writer(f)
    writer.writerow(header)
'''

In [ ]:
'''
def append_missing(link):
    link=missing_links[i]
    headers= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36"}
    page=requests.get(link,headers=headers,cookies={'lc-acbeg':'en_AE'},timeout=30,verify=False)
    soup1= BeautifulSoup(page.content,'html.parser')
    soup2=BeautifulSoup(soup1.prettify(),'html.parser')
    title = soup2.find(id='productTitle')
    brand = soup2.find('a',attrs={"class": "a-link-normal", "id":"bylineInfo"})
    price = soup2.find("span", {"id": "tp_price_block_total_price_ww"})
    today = datetime.date.today()
    rating = soup2.find('span',attrs={"class": "a-icon-alt"})
    num_of_ratings = soup2.find('span',attrs={"id":"acrCustomerReviewText"})
    category = 'N/A'
    data=[title,brand,category,price,today,rating, num_of_ratings,link]
    with open(filename,'a+',newline='',encoding='UTF8') as f:
        writer=csv.writer(f)
        writer.writerow(data)
'''

In [ ]:
'''
for i in range(0,33):
    append_missing(missing_links)
    i+=1
'''

In [ ]:
df_missing_info=pd.read_csv('amazonMissingItems.csv')
df_missing_info.info()

there're 13 missing prices in the entire dataset we need to delete them and then add the new info

In [ ]:
df_missing_info = df_missing_info[df_missing_info['price'].notna()]
df_missing_info.info()

we create a db with sql to join the tables and get the full info for the 20 fixed rows and delete the 33 rows with missing data to get a total of 582 rows.

In [ ]:
cnn = sqlite3.connect('amazonKitchen.db')

In [ ]:
df.to_sql('fullAmazonData', cnn, index=False)
df_missing_info.to_sql('missingAmazonData', cnn, index=False)

In [ ]:
%load_ext sql
%sql sqlite:///amazonKitchen.db

it's easier to join the tables on sql so we use it to join the two tables to correctly assign the categories

In [ ]:
df_missing_info=pd.read_sql('''SELECT f.title, m.brand, f.category, m.price, m.date, m.rating, m.num_of_ratings, m.link
 FROM missingAmazonData m
 JOIN fullAmazonData f
 on m.link=f.link
 ''', con=cnn)
df_missing_info

In [ ]:
df=pd.concat([df,df_missing_info], ignore_index=True)
df.info()

In [ ]:
df = df[df['brand'].notna()]
df = df[df['price'].notna()]
df.info()

In [ ]:

df.to_csv('amazonKitchen.csv', index=False)
df.to_sql('amazonKitchen', cnn, index=False)


In [ ]:
df=pd.read_csv('amazonKitchen.csv')
df.info()

In [ ]:
df.duplicated().sum()

In [ ]:
df.sample(5)

now after we made our data complete, we need to remove the html tags from our columns and make the numeric columns become suitable for numeric analysis.

In [ ]:
def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result
df['title']=df['title'].apply(lambda cw : remove_tags(cw))
df['brand']=df['brand'].apply(lambda cw : remove_tags(cw))
df['price']=df['price'].apply(lambda cw : remove_tags(cw))
df['rating']=df['rating'].apply(lambda cw : remove_tags(cw))
df['num_of_ratings']=df['num_of_ratings'].apply(lambda cw : remove_tags(cw))

In [ ]:
df['title'] = df['title'].str.replace('\n', '', regex=True).str.strip()
df['brand'] = df['brand'].str.replace('\n', '', regex=True).str.strip()
df['price'] = df['price'].str.replace('\n', '', regex=True).str.strip()
df['rating'] = df['rating'].str.replace('Posted ', '', regex=True).str.strip()
df['num_of_ratings'] = df['num_of_ratings'].str.replace(' -', '', regex=True).str.strip()

In [ ]:
df.sample(5)

after checking on excel, there're some brands that are typed more than one way we need to edit them in the next cell.

In [ ]:
df['brand'] = df['brand'].str.replace('Brand: ', '', regex=True).str.strip()
df['brand'] = df['brand'].str.replace('العلامة التجارية: ', '', regex=True).str.strip()
df['brand'] = df['brand'].str.replace('&amp;', '&', regex=True).str.strip()
df['brand'] = df['brand'].str.replace('Phlips', 'PHILIPS', regex=True).str.strip()
df['brand'] = df['brand'].str.replace('Philips Domestic Appliances', 'PHILIPS', regex=True).str.strip()
df['brand'] = df['brand'].str.replace('Philips Kitchen Appliances', 'PHILIPS', regex=True).str.strip()
df['brand'] = df['brand'].str.replace('MediaTech', 'Media Tech', regex=True).str.strip()
df['brand'] = df['brand'].str.replace("De'Longhi", "DeLonghi", regex=True).str.strip()
df['brand'] = df['brand'].str.replace("Touch El Zenouki", "Touch Elzenouki", regex=True).str.strip()

In [ ]:
df['num_of_ratings'] = df['num_of_ratings'].str.replace('ratings', '', regex=True).str.strip()
df['num_of_ratings'] = df['num_of_ratings'].str.replace('rating', '', regex=True).str.strip()
df['num_of_ratings'] = df['num_of_ratings'].str.replace('تقييم', '', regex=True).str.strip()
df['num_of_ratings'] = df['num_of_ratings'].str.replace(',', '', regex=True).str.strip()
df['num_of_ratings'] = pd.to_numeric(df['num_of_ratings'])
df.sample(5)

In [ ]:
df['rating'] = df['rating'].str.replace(' out of 5 stars', '', regex=True).str.strip()
df['rating'] = df['rating'].str.replace('من 5 نجوم', '', regex=True).str.strip()
df['rating'] = pd.to_numeric(df['rating'])
df.sample(5)

In [ ]:
df['price'] = df['price'].str.replace('EGP', '', regex=True).str.strip()
df['price'] = df['price'].str.replace(',', '', regex=True).str.strip()
df['price'] = df['price'].str.replace(' ', '', regex=True).str.strip()
df['price'] = df['price'].str.replace('.\u200e', '', regex=True).str.strip()
df['price'] = df['price'].str[:6]
df['price'] = pd.to_numeric(df['price'])
df.tail(5)

In [ ]:
df['date'] = pd.to_datetime(df['date'], dayfirst=True)
df.info()

In [ ]:
df.sample(5)

after deleting all the data and cleaning it we save the clean dataset to the db and a scv file.

In [ ]:
df.to_csv('amazonKitchenCleaned.csv', index=False)
df.to_sql('amazonKitchenCleaned', cnn, index=False)

# data_analysis

**the main questions**
- who are our main brand competitors either overall or in each category?
- what are the most succefull products in each category and what are their charactaristics?

In [ ]:
df=pd.read_csv('amazonKitchenCleaned.csv', index_col=False)
cnn = sqlite3.connect('amazonKitchen.db')
%load_ext sql
%sql sqlite:///amazonKitchen.db
df.head()

In [ ]:
df.head()

**Q1** who are our main brand competitors either overall or in each category? <br>
to answer this question we type a script to find the best 35 companies in the number of ratings (as it reflects the number of sales) and the count of its products in each category.

In [ ]:
df.groupby(['category','brand']).agg(mean_num_of_ratings=('num_of_ratings','mean'), 
sum_num_of_ratings=('num_of_ratings','sum'), 
count_of_links=('link','count')).nlargest(columns=['sum_num_of_ratings', 'count_of_links'],n=35, keep='all').sort_values(by=['category', 'sum_num_of_ratings', 'count_of_links'], 
ascending=[True,False, False])

- We can find that Black & Decker appeared in 3 categories as a main competetor with a variety of products so we should consider them.
- Braun also appeared in more than one category, Philips has 17 air fryers so it has many products in the air fryers market.
- in some categories it seems like there's a brand that has a great number of products and has great number of ratings with a great difference than the next compitetors such as: Bosch in the mixers market, Delonghi in the espresso machines, and braun in the blenders market.
- Ninja and hario had one product that got them in the list for the top compitetors in the air fryers and electric kettles markets so we should study these products carefully.

**Q2** who are our main brand competitors either overall or in each category? <br>
to answer this question we need to see the products with the most number of ratings in each category.

In [ ]:
plt.figure(figsize=(20,10))
sns.boxplot(data=df, x="category", y="num_of_ratings", hue="category", palette="dark")

the outliers show the products with the most number of ratings which reflects the most number of sales.

In [ ]:
class Quantile:
    def __init__(self, q):
        self.q = q
        
    def __call__(self, x):
        return x.quantile(self.q)
        # Or using numpy
        # return np.quantile(x.dropna(), self.q)

In [ ]:
df.groupby(['category']).agg(mean_num_of_ratings=('num_of_ratings','mean'), median_num_of_ratings=('num_of_ratings','median'), quantile_75=('num_of_ratings',Quantile(0.75)), quantile_95=('num_of_ratings',Quantile(0.95)))

In [ ]:
df_top_products=df.loc[(df['num_of_ratings'] >= 407.80) & (df['category'] == 'air fryers')]
df_top_products

In [ ]:
df_top_products=df_top_products.append([df.loc[(df['num_of_ratings'] >= 1534.75) & (df['category'] == 'blenders')], 
df.loc[(df['num_of_ratings'] >= 198.15) & (df['category'] == 'electric kettles')],
df.loc[(df['num_of_ratings'] >= 3748.15) & (df['category'] == 'espresso machines')],
df.loc[(df['num_of_ratings'] >= 1432.00) & (df['category'] == 'food processors')],
df.loc[(df['num_of_ratings'] >= 1150.00) & (df['category'] == 'mixers')],
df.loc[(df['num_of_ratings'] >= 1373.50) & (df['category'] == 'toasters')]], ignore_index=True)

In [ ]:
df_top_products.info()

there're 33 products that has a number of ratings more than the other 95% products in their category.

In [ ]:
df_top_products

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20,10))
sns.boxplot(data=df_top_products, x="category", y="num_of_ratings", hue="category", palette="dark", ax=axes[0])
sns.boxplot(data=df_top_products, x="category", y="price", hue="category", palette="dark", ax=axes[1])
fig.tight_layout()

- we can see that the air fryers market has the highest prices among all the categories as it varies between around 8000 and 12000 EGP.
- the espresso machines market comes in the second place with a little variety and the prices are around 7000 EGP.
- after that the other markets are close to each other around 1000 and 3500 EGP prices except for the food processors with a range between 6000 and 2000 EGP. 

In [ ]:
df_top_products.link[1]

In [ ]:
top_products=list(df_top_products.link)
len(top_products)

we scrape the description for each product in our list so that we can know more about each one of them.

In [ ]:
""" filename='amazonTopProducts.csv'
header=['link','description'] """

In [ ]:
""" with open(filename,'w',newline='',encoding='UTF8') as f:
    writer=csv.writer(f)
    writer.writerow(header) """

In [ ]:
""" def append_top(link):
    link=top_products[i]
    headers= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36"}
    page=requests.get(link,headers=headers,cookies={'lc-acbeg':'en_AE'},timeout=30,verify=False)
    soup1= BeautifulSoup(page.content,'html.parser')
    soup2=BeautifulSoup(soup1.prettify(),'html.parser')
    description = soup2.find(id='productDescription_feature_div')
    data=[link,description]
    with open(filename,'a+',newline='',encoding='UTF8') as f:
        writer=csv.writer(f)
        writer.writerow(data) """

In [ ]:
""" for i in range(0,33):
    append_top(top_products)
    i+=1 """

In [ ]:
df_top_products_description=pd.read_csv('amazonTopProducts.csv')
df_top_products_description.info()

In [ ]:
def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result
df_top_products_description['description']=df_top_products_description['description'].apply(lambda cw : remove_tags(cw))
df_top_products_description.head()

In [ ]:
df_top_products_description['description'] = df_top_products_description['description'].str.replace('\n', '', regex=True).str.strip()
df_top_products_description.head()

In [ ]:
df_top_products_description['description'] = df_top_products_description['description'].apply(lambda st: st[st.find("Product "):st.find("#productDescription")])
df_top_products_description.head()

In [ ]:
df_top_products_description.link[32]

In [ ]:
df_top_products_description.description[32]

In [ ]:
""" df_top_products.to_sql('amazonTopProducts', cnn, index=False)
df_top_products_description.to_sql('amazonTopProductsDescription', cnn, index=False) """

In [ ]:
df_top_products=pd.read_sql('''SELECT t.title, t.brand, t.category, t.price, t.date, t.rating, t.num_of_ratings, t.link, d.description
 FROM amazonTopProducts t
 JOIN amazonTopProductsDescription d
 on t.link=d.link
 ''', con=cnn)
df_top_products

In [ ]:
""" df_top_products.to_sql('amazonFinalTopProducts', cnn, index=False)
df_top_products.to_csv('amazonFinalTopProducts.csv', index=False) """

In [ ]:
df_top_products=pd.read_csv('amazonFinalTopProducts.csv')
df_top_products

Finally we have a dataset ready for the best products with their description that we can study to get an idea of what an ideal product is.

# conclusions

- the project used beautifulsoup to scrape the amazon website in egypt for 7 categories in the kitchen appliances market.
- after the scraping we used SQL and pandas to clean the dataset and get ready for the analysis.
- the analysis was performed using seaborn, pandas as SQL to answer the business questions and the results are:
    - We concluded that Black & Decker appeared in 3 categories as a main competetor with a variety of products so we should consider them.
    - Braun also appeared in more than one category, Philips has 17 air fryers so it has many products in the air fryers market.
    - in some categories it seems like there's a brand that has a great number of products and has great number of ratings with a great difference than the next compitetors such as: Bosch in the mixers market, Delonghi in the espresso machines, and braun in the blenders market.
    -  Ninja and hario had one product that got them in the list for the top compitetors in the air fryers and electric kettles markets so we should study these products carefully.
    - We can see that the air fryers market has the highest prices among all the categories as it varies between around 8000 and 12000 EGP.
    - the espresso machines market comes in the second place with a little variety and the prices are around 7000 EGP.
    - after that the other markets are close to each other around 1000 and 3500 EGP prices except for the food processors with a range between 6000 and 2000 EGP.
    - the top products were saved in a seperate file with their discription and we can track their performance overtime with append_product function to track the ratings and the price.